<a href="https://colab.research.google.com/github/AlexNedyalkov/AladdinPerssonPyTorchTuorial/blob/main/01_Simple_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

import os

In [ ]:
def check_accuracy(model, loader):
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

# 1. Set up Device Using Device-Agnostic Code



In [ ]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_workers = os.cpu_count()

# 2. Create model

In [ ]:
class FNN(nn.Module):

  def __init__(self, in_features, hidden_layers, num_classes) -> None:
    super().__init__()
    self.linear1 = nn.Linear(in_features = in_features, out_features = hidden_layers)
    self.linear2 = nn.Linear(in_features = hidden_layers, out_features = num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.linear1(x))
    x = self.linear2(x)

    return x


# 3. Set up hyper parameters

In [ ]:
batch_size = 32
lr = 0.001
epochs = 10
hidden_layers = 50

# 4. Get Data

In [ ]:
train_dataset = datasets.CIFAR10(
                        root = 'data/',
                        download = True,
                         train = True,
                         transform = transforms.ToTensor())

test_dataset = datasets.CIFAR10(
                        root = 'data/',
                        download = True,
                         train = False,
                         transform = transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers = num_workers,
)

test_loader = DataLoader(
    dataset = test_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers = num_workers,
)

# 5. Initialize network

In [ ]:
X, y = next(iter(train_loader))

input_size = X.shape[-1] * X.shape[-2] * X.shape[-3]
num_classes = 10

X.shape

torch.Size([32, 3, 32, 32])

In [ ]:
model = FNN(in_features = input_size,
            hidden_layers=hidden_layers,
            num_classes = num_classes).to(device)

In [ ]:
optim = torch.optim.Adam(model.parameters(), lr = lr)
loss_fn = torch.nn.CrossEntropyLoss()

# 6. Train

In [ ]:
for epoch in range(epochs):
  print(f'Epoch {epoch}')
  for X,y in train_loader:

    X = X.to(device)
    y = y.to(device)
    # Get to correct shape
    X = X.reshape(batch_size, -1)

    # forward pass
    y_logit = model(X)
    loss = loss_fn(y_logit, y)

    # backward
    optim.zero_grad()
    loss.backward()
    optim.step()


Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


# 7. Check Performance

In [ ]:
# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set: {check_accuracy(loader = train_loader, model = model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(loader = test_loader, model = model)*100:.2f}")

Accuracy on training set: 35.92
Accuracy on test set: 35.25
